## Load the data

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


train = pd.read_csv('../../data/train.tsv', sep='\t')
test = pd.read_csv('../../data/test.tsv', sep='\t')
train.shape

/home/isaac/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


(156060, 4)

We load more data to train our word2vec models in order to make it understand the relationship among words better in a general sense

In [2]:

# Read data from files 
BPtrain = pd.read_csv( "labeledTrainData.tsv", header=0, delimiter="\t", quoting=3 )
BPtest = pd.read_csv( "testData.tsv", header=0, delimiter="\t", quoting=3 )
BP_unlabeled_train = pd.read_csv( "unlabeledTrainData.tsv", header=0, delimiter="\t", quoting=3 )

# Verify the number of reviews
print "Read %d labeled train reviews, %d labeled test reviews, " \
 "and %d unlabeled reviews\n" % (BPtrain["review"].size,  BPtest["review"].size, BP_unlabeled_train["review"].size )


Read 25000 labeled train reviews, 25000 labeled test reviews, and 50000 unlabeled reviews



## Simple preprossing date (cleaning data)

We use some nice libraries like beautifulsoup, regular experssion(re) to remove some non alphabetical words in order to make the data "nicer" for the model to train later. This function will take a phrase or sentence and break into a list of individual words.

In [3]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def phrase_to_wordlist(review, remove_stopwords=False):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    phrase_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    phrase_text = re.sub("[^a-zA-Z]"," ", phrase_text)
    #
    # 3. Convert words to lower case and split them
    words = phrase_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return words

In this rotten reviews data set, all the data are organized into units of phrases or sentences already. However, for some additional data set we loaded previously, they are a body of String data without being broken down by language natural tokens yet. So in this following function, we use a method from natural language processsing library(nltk) implemented by standford university, tokenizer to split paragraph into sentences.  

In [4]:
# Download the punkt tokenizer for sentence splitting
import nltk.data

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    review = review.decode('unicode_escape').encode('ascii', 'ignore')
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( phrase_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [5]:
from sklearn.cross_validation import train_test_split
train, cv = train_test_split(train, test_size=0.3)

trainPhrases = []
for phrase in train.Phrase:
    trainPhrases.append(phrase_to_wordlist(phrase))
print len(trainPhrases)

cvPhrases = []
for phrase in cv.Phrase:
    cvPhrases.append(phrase_to_wordlist(phrase))
print len(cvPhrases)

testPhrases = []
for phrase in test.Phrase:
    testPhrases.append(phrase_to_wordlist(phrase))
print len(testPhrases)

109242
46818
66292


/home/isaac/anaconda/lib/python2.7/site-packages/bs4/__init__.py:182: UserWarning: "." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  '"%s" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.' % markup)


In [6]:
sentences = []  # Initialize an empty list of sentences

print "Parsing sentences from training set"
for review in BPtrain["review"]:
    sentences += review_to_sentences(review, tokenizer)

print "Parsing sentences from unlabeled set"
for review in BP_unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)


Parsing sentences from training set
Parsing sentences from unlabeled set

/home/isaac/anaconda/lib/python2.7/site-packages/bs4/__init__.py:189: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
/home/isaac/anaconda/lib/python2.7/site-packages/bs4/__init__.py:189: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
/home/isaac/anaconda

/home/isaac/anaconda/lib/python2.7/site-packages/bs4/__init__.py:189: UserWarning: "http://jake-weird.blogspot.com/2007/08/beneath.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)


We combine the rotten reviews trainning data and additional data from other moive reviews together for trainning purpose.

In [7]:
phrases = trainPhrases + cvPhrases + testPhrases + sentences

Simply printing out some of the data to double check the format

In [8]:
print sentences[0]

[u'with', u'all', u'this', u'stuff', u'going', u'down', u'at', u'the', u'moment', u'with', u'mj', u'i', u've', u'started', u'listening', u'to', u'his', u'music', u'watching', u'the', u'odd', u'documentary', u'here', u'and', u'there', u'watched', u'the', u'wiz', u'and', u'watched', u'moonwalker', u'again']


In [9]:
print phrases[0:2]

[[u'invisible', u'nearly', u'psychic', u'nuances', u'leaping', u'into', u'digressions', u'of', u'memory', u'and', u'desire'], [u'the', u'rise', u'of', u'hip', u'hop', u'culture', u'in', u'general', u'and', u'the', u'art', u'of', u'scratching', u'lrb', u'or', u'turntablism', u'rrb', u'in', u'particular']]


## Complicated preprocessing data (word2vec model extraction)

> Word2vec, published by Google in 2013, is a neural network implementation that learns distributed representations for  words. Other deep or recurrent neural network architectures had been proposed for learning word representations prior to this, but the major problem with these was the long time required to train the models. Word2vec learns quickly relative to other models.

> Word2Vec does not need labels in order to create meaningful representations. This is useful, since most data in the real world is unlabeled. If the network is given enough training data (tens of billions of words), it produces word vectors with intriguing characteristics. Words with similar meanings appear in clusters, and clusters are spaced such that some word relationships, such as analogies, can be reproduced using vector math. The famous example is that, with highly trained word vectors, "king - man + woman = queen."

-- Kaggle

In [10]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print "Training model..."
model = word2vec.Word2Vec(trainPhrases, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

Couldn't import dot_parser, loading of dot files will not be possible.
Training model...


Simple testing the performance of the word2vec model 

In [11]:
model.most_similar("no")

[(u'plenty', 0.7094558477401733),
 (u'here', 0.6876622438430786),
 (u'reason', 0.6374000906944275),
 (u'doubt', 0.59196537733078),
 (u'nothing', 0.5884739756584167),
 (u'amount', 0.5492249727249146),
 (u'real', 0.5476399660110474),
 (u'problem', 0.5089858174324036),
 (u'harris', 0.5075728297233582),
 (u'substance', 0.4934156835079193)]

Now that we have a trained model with some semantic understanding of words. The word2vec model has a matrix of all the words it learns. Since each word is a vector in 300-dimensional space, we can use vector operations to combine the words in each phrase or sentence review. One method we are going to try is to simply average the word vectors in a given review.

## Vector Averaging

In [12]:
def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    if nwords > 0:
        featureVec = np.divide(featureVec,nwords)
        
    return featureVec

def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0.
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
       #
       # Print a status message every 1000th review
       if counter%1000. == 0.:
           print "Review %d of %d" % (counter, len(reviews))
       # 
       # Call the function (defined above) that makes average feature vectors
       reviewFeatureVecs[counter] = makeFeatureVec(review, model, \
           num_features)
       #
       # Increment the counter
       counter = counter + 1.
    return reviewFeatureVecs


In [ ]:
trainVecs = getAvgFeatureVecs(trainPhrases, model, 300)
cvVecs = getAvgFeatureVecs(cvPhrases, model, 300)
testVecs = getAvgFeatureVecs(testPhrases, model, 300)

Review 0 of 109242
Review 1000 of 109242
Review 2000 of 109242
Review 3000 of 109242
Review 4000 of 109242
Review 5000 of 109242
Review 6000 of 109242
Review 7000 of 109242
Review 8000 of 109242
Review 9000 of 109242
Review 10000 of 109242
Review 11000 of 109242
Review 12000 of 109242
Review 13000 of 109242
Review 14000 of 109242
Review 15000 of 109242
Review 16000 of 109242
Review 17000 of 109242
Review 18000 of 109242
Review 19000 of 109242
Review 20000 of 109242
Review 21000 of 109242
Review 22000 of 109242
Review 23000 of 109242
Review 24000 of 109242
Review 25000 of 109242
Review 26000 of 109242
Review 27000 of 109242
Review 28000 of 109242
Review 29000 of 109242
Review 30000 of 109242
Review 31000 of 109242
Review 32000 of 109242
Review 33000 of 109242
Review 34000 of 109242
Review 35000 of 109242
Review 36000 of 109242
Review 37000 of 109242
Review 38000 of 109242
Review 39000 of 109242
Review 40000 of 109242
Review 41000 of 109242
Review 42000 of 109242
Review 43000 of 109242
R

## Train Model

After creating the feature vectors to extract the information, we are going to load random forest model from sklearn to train with th precessed data.

In [ ]:
# Fit a random forest to the training data, using 100 trees
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators = 100)

print "Fitting a random forest to labeled training data..."
forest = forest.fit(trainVecs, train.Sentiment)

# Predict using training data
cv_pred = forest.predict(cvVecs)
results_cv = pd.DataFrame({
    'PhraseId': cv.PhraseId,
    'Predicted': cv_pred,
    'Sentiment': cv.Sentiment
})
results_cv.to_csv('results_train.csv', index=False, quoting=3)

# Test & extract results 
result = forest.predict(testVecs)

# Write the test results 
results_test = pd.DataFrame({
    'PhraseId': test.PhraseId,
    'Sentiment': result
})
results_test.to_csv("results_test.csv", index=False, quoting=3)

## Clustering 

Another possible approach is to exploit the similarity of words within a cluster. we can do this by using a clustering algorithm such as K-Means.

In [ ]:
from sklearn.cluster import KMeans
import time

start = time.time() # Start time

# Set "k" (num_clusters) to be 1/5th of the vocabulary size, or an
# average of 5 words per cluster
word_vectors = model.syn0
num_clusters = word_vectors.shape[0] / 5

# Initalize a k-means object and use it to extract centroids
kmeans_clustering = KMeans( n_clusters = num_clusters )
idx = kmeans_clustering.fit_predict( word_vectors )

# Get the end time and print how long the process took
end = time.time()
elapsed = end - start
print "Time taken for K Means clustering: ", elapsed, "seconds."

In [ ]:
# Create a Word / Index dictionary, mapping each vocabulary word to
# a cluster number                                                                                            
word_centroid_map = dict(zip( model.index2word, idx ))

The following is our cluster results

In [ ]:
# For the first 10 clusters
for cluster in xrange(0,10):
    #
    # Print the cluster number  
    print "\nCluster %d" % cluster
    #
    # Find all of the words for that cluster number, and print them out
    words = []
    for i in xrange(0,len(word_centroid_map.values())):
        if( word_centroid_map.values()[i] == cluster ):
            words.append(word_centroid_map.keys()[i])
    print words

Now we have a cluster assignment for each word, and we can define a function to convert reviews into bags-of-clusters. This works just like Bag of Words but uses semantically related clusters instead of individual words:

In [ ]:
def create_bag_of_centroids( wordlist, word_centroid_map ):
    #
    # The number of clusters is equal to the highest cluster index
    # in the word / centroid map
    num_centroids = max( word_centroid_map.values() ) + 1
    #
    # Pre-allocate the bag of centroids vector (for speed)
    bag_of_centroids = np.zeros( num_centroids, dtype="float32" )
    #
    # Loop over the words in the review. If the word is in the vocabulary,
    # find which cluster it belongs to, and increment that cluster count 
    # by one
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    #
    # Return the "bag of centroids"
    return bag_of_centroids

In [ ]:
# Pre-allocate an array for the training set bags of centroids (for speed)
train_centroids = np.zeros( (train.Phrase.size, num_clusters), \
    dtype="float32" )

# Transform the training set reviews into bags of centroids
counter = 0
for phrase in trainPhrases:
    train_centroids[counter] = create_bag_of_centroids( phrase, \
        word_centroid_map )
    counter += 1

# Repeat for test reviews 
test_centroids = np.zeros(( test.Phrase.size, num_clusters), \
    dtype="float32" )

counter = 0
for phrase in testPhrases:
    test_centroids[counter] = create_bag_of_centroids( phrase, \
        word_centroid_map )
    counter += 1

In [ ]:
# Fit a random forest and extract predictions 
forest = RandomForestClassifier(n_estimators = 100)

# Fitting the forest may take a few minutes
print "Fitting a random forest to labeled training data..."
forest = forest.fit(train_centroids, train.Sentiment)
result = forest.predict(test_centroids)
 
# Write the test results 
output = pd.DataFrame({
    'PhraseId': test.PhraseId,
    'Sentiment': result
})
output.to_csv("BagOfCentroids.csv", index=False, quoting=3)